# First try scraping idealista


provarem de parsejar https://www.idealista.com/venta-viviendas/barcelona-barcelona/
i les següents 10 pàgines. No entrarem en cada pis (tot i que guardarem un enllaç per a si volem parsejar el contingut)

In [1]:
# import pandas as pd

import scrapy
from scrapy.crawler import CrawlerProcess

import logging
import pathlib
import re



In [2]:
class Vivienda(scrapy.Item):
    links = scrapy.Field()
    precio = scrapy.Field()
    precio_ant = scrapy.Field()
    nombre = scrapy.Field()
    desc = scrapy.Field()
    habitaciones = scrapy.Field()
    superficie = scrapy.Field()
    otros = scrapy.Field()

In [3]:
class IdeaSpider(scrapy.Spider):
    name = "idealista"
    start_urls = [
        'https://www.idealista.com/venta-viviendas/barcelona-barcelona/',
    ]
    
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'CLOSESPIDER_PAGECOUNT': 3,
        'DOWNLOAD_DELAY' : 15,
        'FEEDS': {
                    'venta_viviendas.json': {
                        'format': 'json',
                        'encoding': 'utf8',
                        'store_empty': False,
                        'fields': None,
                        'indent': 4,
                        'item_export_kwargs': {
                           'export_empty_fields': False,
                        },
                      },                    
                    },
#         'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, # Used for pipeline 1
#         'FEED_FORMAT':'json',                                 # Used for pipeline 2
#         'FEED_URI': '~/Downloads/idearesult-1.json'                        # Used for pipeline 2
    }
    
    def parse(self, response):
        
        for vivienda in response.xpath('//*[@id="main-content"]/section[1]/article'):
  
            # comprovem que no sigui un anunci
    
            if vivienda.attrib['class'] != 'adv noHover':
                viv = Vivienda()

                viv['links'] = vivienda.xpath('.//div/a/@href').getall()
                viv['precio'] = ''.join(re.findall(r'\d+', vivienda.xpath('.//span [@class="item-price h2-simulated"]/text()').get()))
                if vivienda.xpath('.//span [@class="pricedown_price"]/text()').get():
                    viv['precio_ant'] = ''.join(re.findall(r'\d+', vivienda.xpath('.//span [@class="pricedown_price"]/text()').get()))
                viv['nombre'] = vivienda.xpath('.//div[@class="item-info-container"]/a/@title').get()

                for det, car in zip(vivienda.xpath('.//span [@class="item-detail"]/text()').getall(),
                                    vivienda.xpath('.//span [@class="item-detail"]/small/text()').getall()):
                    if 'hab.' in car:
                        viv['habitaciones'] = int(det)
                    elif 'm²' in car:
                        viv['superficie'] = int(det)
                    else:
                        viv['otros'] = "".join([det, car])

                viv['desc'] = vivienda.xpath('.//div[@class="item-description description"]/p//text()').get().strip().replace("\n", ". ")
                yield viv
            
        # request de la pàgina següent    
        seguent = response.xpath('//*[@id="main-content"]/section[1]/div/ul/li[8]/a/@href')
        if seguent:
            seguent_url = response.urljoin(seguent.get())
            request = scrapy.Request(url=seguent_url, )
            yield request

        


links = ``` l.xpath('.//div/a/@href').getall() ```  
precio = ```l.xpath('.//span [@class="item-price h2-simulated"]/text()').get()```
precio_ant = ```l.xpath('.//span [@class="pricedown_price"]/text()').get()```
nombre = ```l.xpath('.//div[@class="item-info-container"]/a/@title').get()```  
habitaciones =   
superficie =
caracter_princ = 
desc = 


```python 
In[59]: lista[0].xpath('.//div[@class="item-info-container"]/a/@title').get()
Out[59]: "Piso en El Camp d'En Grassot i Gràcia Nova, Barcelona"

    
In [95]: lista[0].xpath('.//span [@class="item-price h2-simulated"]/text()').get()
Out[95]: '480.000'

In [97]: lista[
Out[97]: '\n495.000 €\n'
    
In [98]: lista[0].xpath('.//span [@class="item-detail"]/text()').getall()
Out[98]: ['4 ', '126 ', 'Planta 5ª ']

In [103]: lista[0].xpath('.//span [@class="item-detail"]').getall()
Out[103]: 
['<span class="item-detail">4 <small>hab.</small></span>',
 '<span class="item-detail">126 <small>m²</small></span>',
 '<span class="item-detail">Planta 5ª <small>exterior con ascensor</small></span>']

In [159]: lista[0].xpath('.//span [@class="item-detail"]/text()').getall()
Out[159]: ['4 ', '126 ', 'Planta 5ª ']

In [160]: lista[0].xpath('.//span [@class="item-detail"]//text()').getall()
Out[160]: ['4 ', 'hab.', '126 ', 'm²', 'Planta 5ª ', 'exterior con ascensor']

In [161]: lista[0].xpath('.//span [@class="item-detail"]/small/text()').getall()
Out[161]: ['hab.', 'm²', 'exterior con ascensor']

In [162]: 

In [199]: lista[0].xpath('.//div[@class="item-description description"]/p//text()').get().strip().re
     ...: place("\n", ". ")
Out[199]: 'Amplio y luminoso piso en Gràcia Nova. Luminosa y confortable vivienda de 126 m2 construidos que disfruta de orientación sureste (sol de...'
```


Selector: 
```css
#main-content > section.items-container.discard-animation-core-vitals
```

#main-content > section.items-container.discard-animation-core-vitals > article:nth-child(1) > div > a


In [4]:
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(IdeaSpider)
process.start()

2021-07-17 20:43:07 [scrapy.utils.log] INFO: Scrapy 2.5.0 started (bot: scrapybot)
2021-07-17 20:43:07 [scrapy.utils.log] INFO: Versions: lxml 4.6.3.0, libxml2 2.9.4, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.2.0, Python 3.9.6 | packaged by conda-forge | (default, Jul 11 2021, 03:35:11) - [Clang 11.1.0 ], pyOpenSSL 20.0.1 (OpenSSL 1.1.1k  25 Mar 2021), cryptography 3.4.7, Platform macOS-11.4-arm64-arm-64bit
2021-07-17 20:43:07 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2021-07-17 20:43:07 [scrapy.crawler] INFO: Overridden settings:
{'CLOSESPIDER_PAGECOUNT': 3,
 'DOWNLOAD_DELAY': 15,
 'LOG_LEVEL': 30,
 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
